## Statistiche descrittive

Consideriamo nuovamente il DataFrame degli [incidenti](http://dati.comune.milano.it/dataset/9f7bcc9c-20a4-4e48-a7cd-99b15ed11102/resource/38d2171d-1067-4252-9f96-02867a2cc617/download/ds177_trafficotrasporti_incidenti_stradali_persone_infortunate_mese_zona_2001-2016.json).

In [12]:
import pandas as pd
incidenti = pd.read_json("data/incidenti.json")

La funzione `describe` permette di iniziare l'esplorazione del dataset, calcolando alcune statistiche essenziali.

In [13]:
incidenti.describe()

,Anno,Feriti,Incidenti,Mese,Morti,Zona
count,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1728.000000
mean,2008.500000,145.287500,108.352083,6.500000,0.515104,5.000000
std,4.610973,65.982854,48.726844,3.452952,0.752125,2.582736
min,2001.000000,22.000000,17.000000,1.000000,0.000000,1.000000
25%,2004.750000,96.000000,73.000000,3.750000,0.000000,3.000000
50%,2008.500000,137.000000,103.000000,6.500000,0.000000,5.000000
75%,2012.250000,188.000000,140.000000,9.250000,1.000000,7.000000
max,2016.000000,557.000000,439.000000,12.000000,4.000000,9.000000


Notare che `count` conta il numero di righe per cui la variabile non è nulla.

## Rimuovere valori mancanti

La presenza di valori mancanti può creare problemi. Per questo motivo possiamo usare `dropna` per rimuovere tutte le righe che contengono valori mancanti.

In [14]:
incidenti_puliti = incidenti.dropna()
incidenti_puliti.head(3)

,Anno,Feriti,Incidenti,Mese,Morti,Zona
1,2001,209,171,1,0,1.0
2,2001,115,91,1,1,2.0
3,2001,187,141,1,1,3.0


Stiamo però attenti che il nuovo DataFrame potrebbe avere caratteristiche diverse dal DataFrame originale. Andiamo a calcolare il numero medio di `Feriti` e di `Incidenti` per i DataFrame `incidenti` e `incidenti_puliti`.

In [15]:
incidenti.describe()[['Feriti', 'Incidenti']].loc['mean']

Feriti       145.287500
Incidenti    108.352083
Name: mean, dtype: float64

In [16]:
incidenti_puliti.describe()[['Feriti', 'Incidenti']].loc['mean']

Feriti       152.524306
Incidenti    113.753472
Name: mean, dtype: float64

## Creare una nuova colonna

Per creare una nuova colonna a partire da altre già esistenti, bisogna scrivere l'espressione corrispondente. Ad esempio, per creare una colonna con il numero di feriti per incidente, bisogna scrivere

In [17]:
incidenti['rapporto'] = incidenti['Feriti'] / incidenti['Incidenti']

Come usuale, è opportuno verificare il risultato dell'operazione, guardando le prime righe del dataset.

In [18]:
incidenti.head(3)

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto
0,2001,69,50,1,1,NaN,1.380000
1,2001,209,171,1,0,1.0,1.222222
2,2001,115,91,1,1,2.0,1.263736


## Assegnare valori ad una colonna

Un caso particolare si ha quando si vuole assegnare lo stesso valore a tutte le righe di un DataFrame. In questo caso assegnamo il valore `0` ad una nuova colonna che chiamiamo `prova`

In [19]:
incidenti['prova'] = 0

In [20]:
incidenti.head(5)

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto,prova
0,2001,69,50,1,1,NaN,1.380000,0
1,2001,209,171,1,0,1.0,1.222222,0
2,2001,115,91,1,1,2.0,1.263736,0
3,2001,187,141,1,1,3.0,1.326241,0
4,2001,154,105,1,0,4.0,1.466667,0


## Convertire una colonna

Possiamo notare che la colonna `Zona` contiene numeri con virgola (*float*). Siccome in realtà `Zona` è un numero intero, convertiamo il numero usando il metodo `astype`. Dobbiamo dare un argomento a `astype` che è un dizionario che associa ad ogni colonna da convertire, il tipo da usare nella conversione.

In [21]:
incidenti_puliti = incidenti_puliti.astype({'Zona': int})
incidenti_puliti.head(3)

,Anno,Feriti,Incidenti,Mese,Morti,Zona
1,2001,209,171,1,0,1
2,2001,115,91,1,1,2
3,2001,187,141,1,1,3


Però, `astype` richiede che i valori da convertire siano tutti non nulli: per questo motivo abbiamo lavorato su `incidenti_puliti`, invece che su `incidenti`. 

Cosa succede se si eseguono le stesse operazioni su `incidenti`?

## Operare solo su alcune righe

In [22]:
incidenti = incidenti.astype({'Zona': int})

ValueError: Cannot convert non-finite values (NA or inf) to integer

La presenza di valori mancanti rende impossibile l'esecuzione di `astype`. Possiamo sfruttare il fatto che `incidenti` e `incidenti_puliti` hanno lo stesso indice.

In [23]:
incidenti.loc[:, 'Zonaint'] = incidenti_puliti.loc[:, 'Zona']
incidenti.head(3)

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto,prova,Zonaint
0,2001,69,50,1,1,NaN,1.380000,0,NaN
1,2001,209,171,1,0,1.0,1.222222,0,1.0
2,2001,115,91,1,1,2.0,1.263736,0,2.0


La nuova colonna aggiunta (il risultato di `incidenti_puliti.loc[:, 'Zona'].astype(int)`) contiene solo alcune delle righe di `incidenti`. Ma pandas sfrutta gli indici di entrambi i DataFrame per aggiungere la colonna, mettendo valori mancanti dove non trova un indice corrispondente. Ad esempio, `indici_puliti` non ha una riga con indice `0`.
Notare che `Zonaint` non ha numeri interi. Ciò è dovuto alla presenza di valori nulli (`NaN`). Per avere interi dobbiamo eliminarli.

## Operare solo su alcune righe.

Se non si vuole passare per un DataFrame intermedio come `incidenti_puliti`, diventa necessario estendere la chiamata a `loc` esplicitando anche una selezione delle righe, per invocare `astype(int)` solo sulle righe in cui la zona non sia mancante.
Questo passo permette anche di gestire adeguatamente la zona mancante, che verrà posta a *0*.

In [24]:
incidenti['Zona_int'] = 0
incidenti.loc[incidenti['Zona'].notnull(), 'Zona_int'] = \
incidenti.loc[incidenti['Zona'].notnull(), 'Zona'].astype(int)
incidenti.head(3)

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto,prova,Zonaint,Zona_int
0,2001,69,50,1,1,NaN,1.380000,0,NaN,0
1,2001,209,171,1,0,1.0,1.222222,0,1.0,1
2,2001,115,91,1,1,2.0,1.263736,0,2.0,2


## Creare una categoria

Una colonna viene chiamata *categoria* se i suoi valori permettono di classificare le righe.
Adesso andremo a creare una nuova variabile `tipo` che ha tre valori:

*  `a` se il rapporto fra il numero di feriti e il numero di incidenti è minore di 1.3;
*  `b` se il rapporto è fra 1.3 (incluso) e 1.4 (escluso);
*  `c` se il rapporto è maggiore o uguale a 1.4;

Il modo più semplice, ma più lungo, è simile a quanto fatto in precedenza: per ogni valore possibile di `tipo`, si selezionano le righe e si cambia il valore solo di tali righe.

In [25]:
incidenti.loc[incidenti['rapporto'] < 1.3, 'tipo'] = 'a'
incidenti.loc[(incidenti['rapporto'] >= 1.3) & (incidenti['rapporto'] < 1.4), 'tipo'] = 'b'
incidenti.loc[incidenti['rapporto'] >= 1.4, 'tipo'] = 'c'
incidenti.head()

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto,prova,Zonaint,Zona_int,tipo
0,2001,69,50,1,1,NaN,1.380000,0,NaN,0,b
1,2001,209,171,1,0,1.0,1.222222,0,1.0,1,a
2,2001,115,91,1,1,2.0,1.263736,0,2.0,2,a
3,2001,187,141,1,1,3.0,1.326241,0,3.0,3,b
4,2001,154,105,1,0,4.0,1.466667,0,4.0,4,c


## Cancellare una colonna

In [26]:
del incidenti['Zonaint']

In [27]:
incidenti.head(3)

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto,prova,Zona_int,tipo
0,2001,69,50,1,1,NaN,1.380000,0,0,b
1,2001,209,171,1,0,1.0,1.222222,0,1,a
2,2001,115,91,1,1,2.0,1.263736,0,2,a


# Funzioni

## Funzioni aggregate (Serie)

Alcune funzioni possono essere applicate su una Serie intera. Ad esempio la funzione `len` permette di ottenere il numero di righe

In [28]:
len(incidenti)

1920

Sicuramente più interessanti sono le funzioni che calcolano minimo, massimo o la somma. Queste funzioni sono applicate a Serie.

In [29]:
incidenti['Feriti'].sum()

278952

In [30]:
incidenti['Feriti'].min()

22

In [31]:
incidenti['Feriti'].max()

557

Di particolare rilievo è che eventuali valori mancanti vengono gestiti correttamente (vengono ignorati)

## Funzioni aggregate (DataFrame)

Le stesse funzioni aggregate, se vengono invocate su un DataFrame, vengono applicate su tutte le colonne, incluse quelle che non contengono valori numerici.

Il risultato è sempre una Serie.

In [32]:
incidenti.min()

Anno         2001
Feriti         22
Incidenti      17
Mese            1
Morti           0
Zona            1
rapporto        1
prova           0
Zona_int        0
tipo            a
dtype: object

Se si vuole applicare la funzione solo alle colonne numeriche, bisogna usare l'opzione `numeric_only`.

In [33]:
incidenti.min(numeric_only = True)

Anno         2001.0
Feriti         22.0
Incidenti      17.0
Mese            1.0
Morti           0.0
Zona            1.0
rapporto        1.0
prova           0.0
Zona_int        0.0
dtype: float64

## Raggruppare righe

Talvolta il fatto che più righe condividano uno stesso valore può essere sfruttare per costruire gruppi di righe, che poi possono essere analizzati separatamente.

Ad esempio, nel dataset `incidenti`, vogliamo contare quanti incidenti abbiamo avuto per ogni zona.
Ciò richiede di raggruppare con una `groupby`.

In realtà andiamo a raggruppare per `Zona_int` solo per motivi estetici (non è gradevole vedere una zona con il punto decimale).

In [34]:
incidenti.groupby('Zona_int')

Il risultato di una `groupby` è un oggetto opaco, che può essere sfruttato tramite operatori su gruppi. 
Pertanto da solo non serve a molto. Ma diventa molto utile se abbinato a `count`, `min`, `max`, `mean`, `sum`

## Raggruppare righe con operatori

Per contare il numero di incidenti, dobbiamo sommare il valore della colonna `Incidenti`, distinguendo per ogni zona.


Notare in questo caso che le righe che hanno *Zona* mancante non vengono considerate nei raggruppamenti.

In [35]:
incidenti.groupby('Zona_int')['Incidenti'].sum()

Zona_int
0    11470
1    24846
2    13958
3    25470
4    20248
5    20000
6    17524
7    21432
8    25696
9    27392
Name: Incidenti, dtype: int64

## Estrarre un gruppo

Per estrarre un singolo gruppo si può usare il metodo `get_group`. Ad esempio, per estrarre tutti i dati relativi al gruppo della zona 3:

In [36]:
incidenti.groupby('Zona_int').get_group(3)

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto,prova,Zona_int,tipo
3,2001,187,141,1,1,3.0,1.326241,0,3,b
13,2001,200,147,2,1,3.0,1.360544,0,3,b
23,2001,260,178,3,1,3.0,1.460674,0,3,c
33,2001,250,177,4,0,3.0,1.412429,0,3,c
43,2001,287,207,5,1,3.0,1.386473,0,3,b
53,2001,315,211,6,0,3.0,1.492891,0,3,c
63,2001,301,223,7,2,3.0,1.349776,0,3,b
73,2001,167,112,8,0,3.0,1.491071,0,3,c
83,2001,261,182,9,0,3.0,1.434066,0,3,c
93,2001,311,225,10,1,3.0,1.382222,0,3,b


## Operatori aggregati

L'elenco delle funzioni che possono essere applicate ad una `groupby` sono chiamati operatori aggregati. Sono:

*  `mean()` calcola il valore medio
*  `median()` calcola il valore mediano
*  `sum()`  calcola la somma
*  `size()` calcola il numero di elementi dei gruppi
*  `count()` calcola il numero di elementi non mancanti nei gruppi
*  `std()` calcola la deviazione standard
*  `var()` calcola la varianza
*  `sem()` calcola l'errore standard della media
*  `describe()` calcola le statistiche descrittive
*  `first()` calcola il primo valore di ogni gruppo
*  `last()` calcola l'ultimo valore di ogni gruppo
*  `nth()` estrae alcuni valori
*  `min()` calcola il minimo valore di ogni gruppo
*  `max()` calcola il massimo valore di ogni gruppo

## Size e Count

Ricordiamo che `count` esclude dal conteggio i valori mancanti, mentre `size` include i valori mancanti.
Andiamo a confrontare i risultati di queste operazioni, considerano solo la colonna `Zona`, visto che è l'unica a contenere valori mancanti.

Chiaramente ci aspettiamo che `count` restituisca valori minori di quelli calcolati da `size`.

In [37]:
incidenti.groupby('Anno')['Zona'].size()

Anno
2001    120
2002    120
2003    120
2004    120
2005    120
2006    120
2007    120
2008    120
2009    120
2010    120
2011    120
2012    120
2013    120
2014    120
2015    120
2016    120
Name: Zona, dtype: int64

In [38]:
incidenti.groupby('Anno')['Zona'].count()

Anno
2001    108
2002    108
2003    108
2004    108
2005    108
2006    108
2007    108
2008    108
2009    108
2010    108
2011    108
2012    108
2013    108
2014    108
2015    108
2016    108
Name: Zona, dtype: int64

## Raggruppare su più colonne

Possiamo utilizzare più di una colonna per raggruppare, fornendo una lista come argomento della `groupby`.

Ad esempio, andiamo a calcolare il numero totale di incidenti, diviso per zona e mese.

In [39]:
incidenti.groupby(['Zona', 'Mese']).sum()['Incidenti']

Zona  Mese
1.0   1       1909
      2       1755
      3       2164
      4       2036
      5       2466
      6       2218
      7       2182
      8        842
      9       2220
      10      2566
      11      2466
      12      2022
2.0   1       1006
      2        996
      3       1149
      4       1232
      5       1343
      6       1310
      7       1323
      8        694
      9       1194
      10      1402
      11      1217
      12      1092
3.0   1       1873
      2       1802
      3       2168
      4       2142
      5       2408
      6       2418
              ... 
7.0   7       1923
      8       1038
      9       1906
      10      2084
      11      1871
      12      1700
8.0   1       1882
      2       1743
      3       2248
      4       2135
      5       2535
      6       2377
      7       2325
      8       1179
      9       2251
      10      2517
      11      2430
      12      2074
9.0   1       2085
      2       1936
      3       2382
 

## Raggruppamenti e indice

Il comportamento di default è che le variabili indicate nella `groupby` diventino l'indice della Serie o del DataFrame risultante.

In [40]:
gruppi = incidenti.groupby(['Zona_int', 'Mese']).sum()
gruppi.index.names

FrozenList(['Zona_int', 'Mese'])

Se invece si vuole che siano nuove colonne del DataFrame risultante, bisogna usare l'opzione `as_index`

In [41]:
gruppi2 = incidenti.groupby(['Zona_int', 'Mese'], as_index = False).sum()
gruppi2.index.names

FrozenList([None])

Controlliamo esplicitamente che `Zona_int` e `Mese` siano colonne del DataFrame `gruppi2`

In [42]:
gruppi2.columns

Index(['Zona_int', 'Mese', 'Anno', 'Feriti', 'Incidenti', 'Morti', 'Zona',
       'rapporto', 'prova'],
      dtype='object')

# Apply e Tabelle pivot

## Apply

Il metodo `apply` permette di applicare una qualunque funzione a tutte le righe (o, più raramente, le colonne) di un  DataFrame o di una Serie. La funzione riceve direttamente una riga del DataFrame.

Una forma più sofisticata permette di applicare una funzione che riceve in input un intero DataFrame.

Come primo esempio, costruiamo una nuova variabile `pericolo` che corrisponde al numero di incidenti, più il numero di feriti moltiplicato per 100. Il primo passo è costruire la funzione che calcola tale indice.

In [43]:
def calcola_p(riga):
    return riga['Incidenti'] + riga['Feriti'] * 100

Poi si può usare la `apply`

In [44]:
incidenti['pericolo'] = incidenti.apply(calcola_p, axis = 1)
incidenti.head()

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto,prova,Zona_int,tipo,pericolo
0,2001,69,50,1,1,NaN,1.380000,0,0,b,6950
1,2001,209,171,1,0,1.0,1.222222,0,1,a,21071
2,2001,115,91,1,1,2.0,1.263736,0,2,a,11591
3,2001,187,141,1,1,3.0,1.326241,0,3,b,18841
4,2001,154,105,1,0,4.0,1.466667,0,4,c,15505


## Creare una categoria con apply

La soluzione che abbiamo visto in precedenza per creare la categoria `tipo` ha due difetti: è fragile (perchè il valore soglia *1.3* compare in due istruzioni diverse, quindi è più probabile scrivere un valore sbagliato) e richiede un'istruzione distinta per ogni valore possibile di `tipo`.

Quindi prima scriviamo una funzione che riceve in input un numero e calcola il valore della categoria.

In [45]:
def categoria(n):
    soglie = [('a', 1.3), ('b', 1.4), ('c', None)]
    for (categoria, soglia) in soglie:
        if soglia is None or n < soglia:
            return categoria

Poi è possibile *applicare* la funzione a `rapporto`

In [46]:
incidenti['tipo'] = incidenti['rapporto'].apply(categoria)
incidenti.head(5)

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto,prova,Zona_int,tipo,pericolo
0,2001,69,50,1,1,NaN,1.380000,0,0,b,6950
1,2001,209,171,1,0,1.0,1.222222,0,1,a,21071
2,2001,115,91,1,1,2.0,1.263736,0,2,a,11591
3,2001,187,141,1,1,3.0,1.326241,0,3,b,18841
4,2001,154,105,1,0,4.0,1.466667,0,4,c,15505


## Differenze

Nelle slide precedenti abbiamo visto due diverse tipologie di utilizzo di apply:

1.  `incidenti.apply(calcola_p, axis = 1)`
2.  `incidenti['rapporto'].apply(categoria)`

Nel caso 1 diventa essenziale l'opzione `axis = 1` che indica a pandas che la funzione deve essere invocata per ogni riga. In questo caso la funzione `calcola_p` riceve un argomento che è la riga di un DataFrame. Tecnicamente la riga è un dizionario con chiavi corrispondenti alle colonne del DataFrame.

Nel caso 2 invece la `apply` viene chiamata su una Serie. In questo caso la funzione `categoria` riceve un singolo valore in input e la parte `axis = 1` non è necessaria.

Un terzo caso prevede invece che la funzione riceva un intero DataFrame

## Tabella pivot

Una tabella pivot viene ottenuta a partire da un DataFrame per riassumere i dati di quest'ultimo.
In particolare, in una tabella pivot sono presenti tre elementi:
*  un insieme di variabili per guidano il raggruppamento di righe
*  una variabile (o un insieme di variabili) che etichetta nuove colonne
*  una funzione per aggregare i valori.

Ad esempio, andiamo a costruire una tabella pivot che riassume per ogni *anno* il numero totale (quindi la *somma) di incidenti (feriti, ecc.), ed in cui si ha una colonna per ogni *zona*.

In [48]:
pd.pivot_table(incidenti, index = 'Anno', columns = ['Zona_int'], values = 'Incidenti', aggfunc = sum)

Zona_int,0,1,2,3,4,5,6,7,8,9
Anno,,,,,,,,,,
2001,605,2207,1171,2176,1782,1693,1409,1828,2200,2387
2002,685,2177,1272,2185,1713,1609,1475,1774,2147,2330
2003,671,2056,1160,2061,1623,1628,1371,1868,2034,2247
2004,682,1958,1101,2155,1579,1623,1287,1650,2051,2230
2005,716,1867,1023,1880,1479,1512,1232,1558,1898,2016
2006,675,1809,1011,1821,1527,1397,1204,1496,1996,2016
2007,574,1723,974,1735,1383,1432,1302,1495,1802,1956
2008,498,1502,916,1715,1322,1309,1132,1425,1747,1763
2009,494,1552,844,1507,1266,1232,1120,1308,1560,1647


## Tabella pivot con medie

Talvolta la funzione per aggregare i valori non è la somma, ma la media. In questo caso si può omettere l'opzione `aggfunc).

In [49]:
pd.pivot_table(incidenti, index = 'Anno', columns = ['Zona_int'], values = 'Incidenti')

Zona_int,0,1,2,3,4,5,6,7,8,9
Anno,,,,,,,,,,
2001,50.416667,183.916667,97.583333,181.333333,148.500000,141.083333,117.416667,152.333333,183.333333,198.916667
2002,57.083333,181.416667,106.000000,182.083333,142.750000,134.083333,122.916667,147.833333,178.916667,194.166667
2003,55.916667,171.333333,96.666667,171.750000,135.250000,135.666667,114.250000,155.666667,169.500000,187.250000
2004,56.833333,163.166667,91.750000,179.583333,131.583333,135.250000,107.250000,137.500000,170.916667,185.833333
2005,59.666667,155.583333,85.250000,156.666667,123.250000,126.000000,102.666667,129.833333,158.166667,168.000000
2006,56.250000,150.750000,84.250000,151.750000,127.250000,116.416667,100.333333,124.666667,166.333333,168.000000
2007,47.833333,143.583333,81.166667,144.583333,115.250000,119.333333,108.500000,124.583333,150.166667,163.000000
2008,41.500000,125.166667,76.333333,142.916667,110.166667,109.083333,94.333333,118.750000,145.583333,146.916667
2009,41.166667,129.333333,70.333333,125.583333,105.500000,102.666667,93.333333,109.000000,130.000000,137.250000


I dati presenti in una tabella pivot possono essere calcolati con una `groupby`, ma la struttura del DataFrame risultante è diversa. Ogni volta bisogna capire se preferiamo la forma ottenuta con la `groupby` (poche colonne, tante righe) o la tabella pivot (meno righe, più colonne).

Di solito la tabella pivot è più semplice da leggere per una persona, ma più complicata da rielaborare.

## Elementi estremi

Di particolare interesse è capire chi ha realizzato un valore estremo (per valore estremo si intende un minimo o un massimo). Ad esempio vogliamo sapere in quale anno si sono verificati più incidenti.

Il primo passo è calcolare il numero di incidenti per ogni anno.

In [50]:
incidenti_anno = incidenti.groupby('Anno').sum()[['Incidenti', 'Feriti', 'Morti']]
incidenti_anno.head(3)

,Incidenti,Feriti,Morti
Anno,,,
2001,17458,23998,86
2002,17367,23843,76
2003,16719,22422,72


Adesso possiamo calcolare il valore massimo della colonna `Incidenti` e selezionare le righe che hanno tale valore.

In [51]:
massimo = incidenti_anno['Incidenti'].max()
massimo

17458

In [52]:
incidenti_anno[incidenti_anno['Incidenti'] == massimo]

,Incidenti,Feriti,Morti
Anno,,,
2001,17458,23998,86


## Elementi estremi 2

Un'alternativa più rapida è fornita dai metodi `idxmax` e `idxmin` che restituiscono il valore dell'*indice* che realizza il valore estremo. Questo valore può essere utilizzato per estrarre la riga che ci interessa.

In [53]:
incidenti_anno['Incidenti'].idxmax()

2001

In [54]:
incidenti_anno.loc[incidenti_anno['Incidenti'].idxmax()]

Incidenti    17458
Feriti       23998
Morti           86
Name: 2001, dtype: int64

**Confronto**

*  La soluzione con `idxmax` è più veloce da scrivere e permette di individuare facilmente l'anno che ci interessa. Ma bisogna ricordare che `idxmax` permette di calcolare un solo valore (in questo caso, un solo anno).
*  La soluzione con la selezione è più complicata. Inoltre per calcolare l'anno che ci interessa, dovremmo fare una nuova estrazione. Il vantaggio è che è corretta anche nel caso in cui due anni diversi realizzano entrambi il massimo valore.

## Idxmax e raggruppamenti

Talvolta ci interessa capire chi ha realizzato un massimo all'interno di ogni gruppo. In altre parole, vogliamo isolare non una sola riga, ma una riga per ogni gruppo.

Ad esempio, per ogni anno vogliamo calcolare in quale mese si sono verificati più incidenti. Il primo passo è costruire un DataFrame che dica per ogni coppia (`Anno`, `Mese`) quanti incidenti si sono verificati.

In [55]:
incidenti_mese = incidenti.groupby(['Anno', 'Mese'], as_index = False).sum()[['Anno', 'Mese', 'Incidenti']]
incidenti_mese

,Anno,Mese,Incidenti
0,2001,1,1233
1,2001,2,1188
2,2001,3,1419
3,2001,4,1369
4,2001,5,1697
5,2001,6,1842
6,2001,7,1657
7,2001,8,848
8,2001,9,1491
9,2001,10,1675


Applicando `idxmax` otteniamo il singolo mese in cui si sono verificati più incidenti, ma noi vogliamo un mese per ogni anno (quindi un mese per il 2001, un mese per il 2002, ecc).

In [56]:
incidenti_mese.loc[incidenti_mese['Incidenti'].idxmax()]

Anno         2001
Mese            6
Incidenti    1842
Name: 5, dtype: int64

Quindi sappiamo che il picco del numero di incidenti si è verificato nel mese di Giugno del 2001. Ma ciò non è quello che desideriamo.

## Idxmax e raggruppamento 2

Raggruppare per anno le righe di `incidenti_mese` è il primo passo: bisogna applicare la funzione `idxmax`.

In [57]:
incidenti_mese.groupby('Anno')['Incidenti'].idxmax()

Anno
2001      5
2002     21
2003     28
2004     45
2005     52
2006     69
2007     76
2008     93
2009    105
2010    112
2011    130
2012    141
2013    153
2014    160
2015    177
2016    189
Name: Incidenti, dtype: int64

Siccome `idxmax` restituisce i valori dell'indice corrispondenti alle righe che realizzano un massimo, bisogna passare per una `loc` per ottenere i mesi corrispondenti.

In [58]:
incidenti_mese.loc[incidenti_mese.groupby('Anno')['Incidenti'].idxmax(), ['Anno', 'Mese']]

,Anno,Mese
5,2001,6
21,2002,10
28,2003,5
45,2004,10
52,2005,5
69,2006,10
76,2007,5
93,2008,10
105,2009,10
112,2010,5
